In [1]:
from warnings import filterwarnings
import pickle
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import gc
from datetime import datetime as dt
filterwarnings('ignore')

In [9]:
target_action = ['sub_car_claim_click', 'sub_car_claim_submit_click',
'sub_open_dialog_click', 'sub_custom_question_submit_click',
'sub_call_number_click', 'sub_callback_submit_click', 'sub_submit_success',
'sub_car_request_submit_click']



df = pd.DataFrame
with open('data\\ga_sessions.pkl', 'rb') as f:
    df = pickle.load(f)
    
with open('data\\ga_hits.pkl', 'rb') as f:
    df_hits = pickle.load(f)    
df_hits['event_action'] = df_hits['event_action'].fillna('no_action')
df_hits['target_event'] = df_hits['event_action'].apply(lambda x: 1 if x in target_action else 0)

df_hits['hit_page_path'] = df_hits.hit_page_path.str.split('cars', 1, expand=True)[1].str.split('?', 1, expand = True)[0]
# df_hits['hit_page_path'].value_counts()

df_target = df_hits[['session_id', 'hit_page_path', 'target_event']].copy()

df_target = df_target[~df_target.duplicated()]

# print(f'count lines before merge: {df.shape[0]}')
df = df.merge(df_target, on='session_id', how='inner')
df.hit_page_path = df.hit_page_path.fillna('without_car')
df.hit_page_path = df['hit_page_path'].apply(lambda a: 1 if a in ['without_car'] else 0)
# del df_hits
# del df_target
# gc.collect()
print(f"shape after merge: {df.shape}")
df.to_csv("data\\stage#1.csv")
df.head()

shape after merge: (3191073, 20)


,session_id,client_id,visit_date,visit_time,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_model,device_screen_resolution,device_browser,geo_country,geo_city,hit_page_path,target_event
0,9055434745589932991.1637753792.1637753792,2108382700.1637753791,2021-11-24,14:36:32,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,NaN,360x720,Chrome,Russia,Zlatoust,1,0
1,905544597018549464.1636867290.1636867290,210838531.1636867288,2021-11-14,08:21:30,1,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,xhoenQgDQsgfEPYNPwKO,IGUCNvHlhfHpROGclCit,mobile,Android,Samsung,NaN,385x854,Samsung Internet,Russia,Moscow,1,0
2,9055446045651783499.1640648526.1640648526,2108385331.1640648523,2021-12-28,02:42:06,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,NaN,360x720,Chrome,Russia,Krasnoyarsk,0,0
3,9055446045651783499.1640648526.1640648526,2108385331.1640648523,2021-12-28,02:42:06,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,NaN,360x720,Chrome,Russia,Krasnoyarsk,1,0
4,9055447046360770272.1622255328.1622255328,2108385564.1622255328,2021-05-29,05:00:00,1,kjsLglQLzykiRbcDiGcD,cpc,NaN,NOBKLgtuvqYWkXQHeYWM,NaN,mobile,None,Xiaomi,NaN,393x786,Chrome,Russia,Moscow,1,0


In [10]:
df['hit_page_path'].value_counts()

0    1733010
1    1458063
Name: hit_page_path, dtype: int64